# Import bibliotek.

In [1]:
import numpy as np
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences # obcina tekst do określonej długości
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten # warstwa Embedding tworzy przestrzeń osadzeń tekstu

# Pobranie, eksploracja i preprocessing danych.

In [2]:
# pobieramy zbiór recenzji filmowych z IMDb

!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
!unzip -q reviews.zip

--2025-03-14 16:41:55--  https://storage.googleapis.com/esmartdata-courses-files/ann-course/reviews.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.207, 172.253.118.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42878657 (41M) [application/x-zip-compressed]
Saving to: ‘reviews.zip’

reviews.zip         100%[===================>]  40.89M  11.8MB/s    in 5.6s    

2025-03-14 16:42:03 (7.24 MB/s) - ‘reviews.zip’ saved [42878657/42878657]



In [3]:
# tak jak w przypadku klasyfikacji obrazów, tworzymy hierarchię katalogów

data_dir = './reviews' # katalog nadrzędny
train_dir = os.path.join(data_dir, 'train') #

train_texts = [] # lista treningowa tekstów
train_labels = [] # lista treningowa etykiet

for label_type in ['neg', 'pos']: # iterujemy po dwóch rodzajach etykiet
    dir_name = os.path.join(train_dir, label_type) # katalog etykiet pozytywnych i negatywnych
    for fname in os.listdir(dir_name): # przechodzimy po każdym tekście z katalogu
        if fname[-4:] == '.txt': # jeżeli plik ma rozszerzenie .txt
            f = open(os.path.join(dir_name, fname)) # otwieramy go
            train_texts.append(f.read()) # dołączamy do listy z tekstami treningowymi
            f.close()
            if label_type == 'neg':
                train_labels.append(0) # 0 - klasa negatywna
            else:
                train_labels.append(1) # 1 - klasa pozytywna

In [4]:
# analogicznie postępujemy w przypadku zbioru testowego

test_dir = os.path.join(data_dir, 'test')

test_texts = []
test_labels = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            test_texts.append(f.read())
            f.close()
            if label_type == 'neg':
                test_labels.append(0)
            else:
                test_labels.append(1)

Etykiety są uporządkowane, więc najpierw po kolei mamy w zbiorze treningowym etykiety negatywne, a potem pozytywne (wyświetlamy po dziesięć pierwszych i ostatnich).

In [5]:
print(train_labels[:10])
print(train_labels[-10:])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [6]:
# podejrzyjmy zawartość pierwszych dziesięciu recenzji

train_texts[:10]

["Bad acting. Bad writing. This was a poorly written film. It's too bad because it had some potential. It's not even close to American Pie or Something about Mary as previous comments might have you believe. Rent it at dollar night from you local video store if you're kind of bored.",
 'You do realize that you\'ve been watching the EXACT SAME SHOW for eight years, right? I could understand the initial curiosity of seeing strangers co-exist on an Island, but you\'d think that after watching unkempt, stink-ladened heroes run roughshod through the bush with an egg on a spoon for half a decade would be enough to get you to commit to something a little more original (and interesting).<br /><br />And I\'m not even speaking of the shows validity which for the record I find questionable. It\'s just hard to suspend disbelief for "Bushy Bill" eating a rat when the entire crew of producers and camera people are housed in an air conditioned make-shift bio-dome sipping frosty mochcinno\'s with moxy

In [7]:
# przeprowadzamy tokenizację

maxlen = 100 # skracamy recenzje do 100 słów
num_words = 10000 # 10000 najczęściej pojawiających się słów
embedding_dim = 100 # wymiar osadzenia

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_texts) # dopasowanie tokenizera na zbiorze treningowym

In [8]:
list(tokenizer.index_word.items())[:20] # wybieramy pierwsze 10 najczęstszych słów

[(1, 'the'),
 (2, 'and'),
 (3, 'a'),
 (4, 'of'),
 (5, 'to'),
 (6, 'is'),
 (7, 'br'),
 (8, 'in'),
 (9, 'it'),
 (10, 'i'),
 (11, 'this'),
 (12, 'that'),
 (13, 'was'),
 (14, 'as'),
 (15, 'for'),
 (16, 'with'),
 (17, 'movie'),
 (18, 'but'),
 (19, 'film'),
 (20, 'on')]

In [9]:
sequences = tokenizer.texts_to_sequences(train_texts) # konwersja tekstu na sekwencje
print(sequences[:3])

[[75, 113, 75, 484, 11, 13, 3, 858, 395, 19, 42, 96, 75, 85, 9, 66, 46, 982, 42, 21, 57, 488, 5, 295, 3264, 39, 139, 41, 1079, 14, 956, 792, 235, 25, 22, 261, 847, 9, 30, 2847, 311, 36, 22, 716, 371, 1127, 44, 332, 240, 4, 1095], [22, 78, 920, 12, 871, 74, 146, 1, 2588, 169, 120, 15, 2306, 150, 205, 10, 97, 388, 1, 2401, 3606, 4, 316, 5074, 998, 1773, 20, 32, 1105, 18, 1383, 101, 12, 100, 146, 7180, 1730, 518, 140, 1, 3432, 16, 32, 8422, 20, 3, 15, 317, 3, 2064, 59, 27, 192, 5, 76, 22, 5, 3513, 5, 139, 3, 114, 50, 201, 2, 218, 7, 7, 2, 143, 21, 57, 1384, 4, 1, 284, 60, 15, 1, 1848, 10, 166, 4588, 42, 40, 251, 5, 4939, 2781, 15, 985, 1883, 3, 4077, 51, 1, 433, 1048, 4, 1177, 2, 367, 81, 23, 8, 32, 942, 94, 6702, 8423, 16, 7, 7, 800, 1, 1268, 130, 10, 89, 456, 41, 131, 81, 39, 65, 6354, 453, 10, 40, 89, 76, 9, 18, 44, 22, 78, 166, 620, 109, 7799, 31, 8424, 81, 10, 1461, 22, 468, 122, 126, 245, 2, 40, 190, 3, 1184, 5, 126, 716, 2640, 1706, 118, 22, 67, 64, 81, 37, 11, 8, 65, 280, 33, 680,

In [10]:
word_index = tokenizer.word_index
print(f'{len(word_index)} unikatowych słów.')

88582 unikatowych słów.


In [11]:
# skracamy recenzje do pierwszych 100 słów

train_data = pad_sequences(sequences, maxlen=maxlen)
train_data.shape

(25000, 100)

In [12]:
train_data[:3] # wyświetlamy ponownie pierwsze trzy recenzje treningowe jako sekwencje

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   75,  113,   75,  484,   11,   13,
           3,  858,  395,   19,   42,   96,   75,   85,    9,   66,   46,
         982,   42,   21,   57,  488,    5,  295, 3264,   39,  139,   41,
        1079,   14,  956,  792,  235,   25,   22,  261,  847,    9,   30,
        2847,  311,   36,   22,  716,  371, 1127,   44,  332,  240,    4,
        1095],
       [  11,    8,   65,  280,   33,  680,   95, 3265,   81,    2,  875,
           4, 2903,   22,   67,  866,  142,    2, 5621,    8,   65, 7181,
        1250,    5, 8926,  995, 6887,   36,    3,    4, 1240,  848, 3331,
         987, 5340,    2,    3,  881, 5229, 5341, 1226,   29,    1,  134,
        6703,   81,   1

In [13]:
# przekształcamy listę etykiet na tablicę NumPy

train_labels = np.asarray(train_labels)
train_labels

array([0, 0, 0, ..., 1, 1, 1])

In [14]:
# przemieszanie próbek

indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

train_data.shape

(25000, 100)

In [15]:
# podział na zbiór treningowy i walidacyjny

training_samples = 15000 # liczba próbek treningowych
validation_samples = 10000 # liczba próbek walidacyjnych

X_train = train_data[:training_samples]
y_train = train_labels[:training_samples]
X_val = train_data[training_samples: training_samples + validation_samples]
y_val = train_labels[training_samples: training_samples + validation_samples]

# Budowa modelu.

In [16]:
model = Sequential()
# warstwa Embedding określa rozmiar danych wejściowych i rozmiar wyjścia
model.add(Embedding(num_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
# kompilacja i trening modelu

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=5,
                    validation_data=(X_val, y_val))

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.6455 - loss: 0.6008 - val_accuracy: 0.8188 - val_loss: 0.4081
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9209 - loss: 0.2191 - val_accuracy: 0.8356 - val_loss: 0.3926
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9913 - loss: 0.0425 - val_accuracy: 0.8242 - val_loss: 0.5191
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9989 - loss: 0.0066 - val_accuracy: 0.8218 - val_loss: 0.6286
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9999 - loss: 5.9515e-04 - val_accuracy: 0.8258 - val_loss: 0.7106


In [18]:
# wykresy metryk - dokładności i straty

def plot_hist(history):
    import pandas as pd
    import plotly.graph_objects as go
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='accuracy vs. val accuracy', xaxis_title='Epoki', yaxis_title='accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='loss vs. val loss', xaxis_title='Epoki', yaxis_title='loss', yaxis_type='log')
    fig.show()

plot_hist(history)

Model szybko zaczyna się przeuczać - o ile metryki na zbiorze treningowym są niemal doskonałe, tak nie widać żadnej poprawy na zbiorze walidacyjnym. Sprawdźmy jeszcze stratę i dokładność modelu na zbiorze testowym.

In [19]:
sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

[0.7214632034301758, 0.8172000050544739]

In [20]:
model.metrics_names

['loss', 'compile_metrics']

# Sieci rekurencyjne.

In [21]:
# SimpleRNN - prosta warstwa rekurencyjna:
# przyjmuje sekwencyjne dane wejściowe (np. tekst lub szereg czasowy).
# przekazuje każdą wartość z sekwencji przez ukryte stany w pętli, gdzie stan ukryty jest aktualizowany na podstawie bieżącego wejścia i poprzedniego stanu.
# wyjście jest uzależnione od przetworzonych danych i poprzednich stanów.

# LSTM - warstwa o długiej pamięci krótkotrwałej
# wykorzystuje specjalne "bramki", które kontrolują przepływ informacji:
# bramka zapominania (Forget Gate): Decyduje, które informacje zostaną zapomniane z poprzednich stanów.
# bramka wejścia (Input Gate): Określa, które nowe informacje będą zapisane w pamięci.
# bramka wyjścia (Output Gate): Kontroluje, które informacje zostaną przekazane do następnych warstw.

from tensorflow.keras.layers import SimpleRNN, LSTM

In [22]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(16)) # dodajemy prostą warstwę zamiast warstwy gęstej
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - accuracy: 0.5794 - loss: 0.6552 - val_accuracy: 0.8089 - val_loss: 0.4393
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8414 - loss: 0.3831 - val_accuracy: 0.8273 - val_loss: 0.3917
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8897 - loss: 0.2811 - val_accuracy: 0.8353 - val_loss: 0.3790
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9208 - loss: 0.2147 - val_accuracy: 0.8459 - val_loss: 0.3763
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9428 - loss: 0.1574 - val_accuracy: 0.8424 - val_loss: 0.4100
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9648 - loss: 0.1053 - val_accuracy: 0.8338 - val_loss: 0.5300
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9765 - loss: 0.0687 - val_accuracy: 0.8291 - val_loss: 0.5341
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9870 - loss: 0.0431 - val_a

In [24]:
# dopiero po wytrenowaniu można wyświetlić podsumowanie sieci z warstwami rekurencyjnymi

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 16)                  │             784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 641,604 (2.45 MB)

 Trainable params: 320,801 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 320,803 (1.22 MB)

In [25]:
# model przeucza się nieco później, ale i tak dość szybko

plot_hist(history)

In [26]:
# sprawdźmy, czy model z warstwą LSTM będzie się uczyć wydajniej

model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(X_val, y_val))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.6520 - loss: 0.6011 - val_accuracy: 0.8337 - val_loss: 0.3732
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8694 - loss: 0.3170 - val_accuracy: 0.8342 - val_loss: 0.3744
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8928 - loss: 0.2578 - val_accuracy: 0.8526 - val_loss: 0.3433
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9182 - loss: 0.2158 - val_accuracy: 0.8421 - val_loss: 0.4052
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9266 - loss: 0.1966 - val_accuracy: 0.8454 - val_loss: 0.4118
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9390 - loss: 0.1718 - val_accuracy: 0.8354 - val_loss: 0.4045
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9414 - loss: 0.1587 - val_accuracy: 0.8479 - val_loss: 0.4028
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9501 - loss: 0.1477 - val_acc

In [28]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 646,308 (2.47 MB)

 Trainable params: 323,153 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 323,155 (1.23 MB)

In [29]:
plot_hist(history)

In [30]:
# ponieważ model zdaje się przeuczać po upływie trzeciej epoki
# ostatni model będzie się trenował właśnie przez tyle epok

model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=3,
                    validation_data=(X_val, y_val))

model.summary()

Epoch 1/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.6546 - loss: 0.6024 - val_accuracy: 0.8250 - val_loss: 0.3957
Epoch 2/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8652 - loss: 0.3223 - val_accuracy: 0.8491 - val_loss: 0.3490
Epoch 3/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8999 - loss: 0.2540 - val_accuracy: 0.8569 - val_loss: 0.3340


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 646,308 (2.47 MB)

 Trainable params: 323,153 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 323,155 (1.23 MB)

In [31]:
# na koniec ewaluujemy model

sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

model.evaluate(X_test, y_test, verbose=0)

[0.34411051869392395, 0.8481600284576416]

Dokładność na zbiorze testowym wyniosła ok. 0,85, czyli wynik zbliżony do tego ze zbioru walidacyjnego.